In [7]:
import numpy as np
import pandas as pd
import math

def load_sheet(file_path, sheet_name):
    return pd.read_excel(file_path, sheet_name=sheet_name)

def process_games_data(df):
    df['Gameweek'] = 'Gameweek ' + (df.index + 1).astype(str)
    conditions = [
        df['Team A Goals'] > df['Team B Goals'],
        df['Team A Goals'] < df['Team B Goals']
    ]
    choices = ['Team A', 'Team B']
    df['Winning Team'] = np.select(conditions, choices, default='Draw')
    df['Total Goals'] = df['Team A Goals'] + df['Team B Goals']
    return df[['Date', 'Gameweek', 'Team A Goals', 'Team B Goals', 'Winning Team', 'Total Goals']]

def merge_data(df1, df2, keys, how='left'):
    return pd.merge(df1, df2, on=keys, how=how)

def fetch_parameters(parameters_data, parameter, game_types):
    return {game_type: parameters_data.loc[parameters_data['Parameter'] == parameter, game_type].values[0] for game_type in game_types}

def get_points(new_points_data, column_name, parameters_data, parameter):
    points = fetch_parameters(parameters_data, parameter, ['5-a-side', '7-a-side', '11-a-side'])
    new_points_data[column_name] = new_points_data['Game Type'].map(points)
    if 'Goals' in column_name:
        new_points_data[column_name] *= new_points_data['Goals']
    return new_points_data

def get_player_position(players_data, position_mapping):
    def position(row):
        game_type = row['Game Type']
        player = row['Player']
        position_col = position_mapping.get(game_type)
        return players_data.loc[players_data['Player'] == player, position_col].values[0] if player in players_data['Player'].values else np.nan
    return position

# Load sheets from Excel
games_data = load_sheet('FantaSPL_V2.3.xlsx', 'Games')
points_data = load_sheet('FantaSPL_V2.3.xlsx', 'Points')
parameters_data = load_sheet('FantaSPL_V2.3.xlsx', 'Parameters')
players_data = load_sheet('FantaSPL_V2.3.xlsx', 'Players')

# Process games data
processed_games_data = process_games_data(games_data)

# Count the number of players for each date
player_counts_per_date = points_data.groupby('Date').agg({'Player': 'nunique'}).reset_index()
player_counts_per_date.columns = ['Date', 'Number of Players']

# Merge dataframes
merged_data = merge_data(processed_games_data, player_counts_per_date, 'Date')

merged_data['Game Type'] = merged_data.apply(lambda row: '5-a-side' if row['Number of Players'] <= 10 else ('7-a-side' if row['Number of Players'] <= 16 else '11-a-side'), axis=1)

# Create a new points dataframe and merge
new_points_data = points_data.copy()
merge_keys = ['Date']
new_points_data = merge_data(new_points_data, merged_data, merge_keys)

# Calculate various points
for col, param in [('Participation Points', 'Participation'), ('Goal Points', 'Goal'), ('Own Goal Points', 'Own Goal'), 
                   ('Penalty Points', 'Penalty'), ('SPL Bonus', 'SPL Bonus'), ('MVP', 'MVP'), ('Friend Referral', 'Friend Referral')]:
    new_points_data = get_points(new_points_data, col, parameters_data, param)

# Get game outcome points
outcome_points = {
    'Draw': fetch_parameters(parameters_data, 'Draw', ['5-a-side', '7-a-side', '11-a-side']),
    'Win': fetch_parameters(parameters_data, 'Win', ['5-a-side', '7-a-side', '11-a-side']),
    'Loss': {'5-a-side': 0, '7-a-side': 0, '11-a-side': 0}
}

new_points_data['Game Outcome Points'] = new_points_data.apply(lambda row: outcome_points['Draw'][row['Game Type']] if row['Winning Team'] == 'Draw' else (outcome_points['Win'][row['Game Type']] if row['Winning Team'] == row['Team'] else outcome_points['Loss'][row['Game Type']]), axis=1)

# Add positions based on match type
position_mapping = {'11-a-side': 'Position 11-a-side', '7-a-side': 'Position 7-a-side', '5-a-side': 'Position 5 a-side'}
new_points_data['Player Position'] = new_points_data.apply(get_player_position(players_data, position_mapping), axis=1)

# Additional functions for computing defensive and midfield scores (as in your code) and apply to `new_points_data` as needed
defense_score_11, defense_score_7, defense_score_5 = 10, 7, 5
goalkeeper_score_11, goalkeeper_score_7, goalkeeper_score_5 = 5, 4, 3

def calculate_defensive_score(row):
    # Extracting necessary data from the row
    game_type = row['Game Type']
    player_position = row['Player Position']
    goals_conceded = row['Team A Goals'] if row['Team'] == 'Team B' else row['Team B Goals']
    
    # Define defensive score based on game type and player position
    if game_type == '11-a-side':
        if player_position == 'Goalkeeper':
            return defense_score_11 - goals_conceded + goalkeeper_score_11
        elif player_position == 'Defender':
            return defense_score_11 - goals_conceded
        elif player_position == 'Midfielder':
            return math.ceil(defense_score_11 - goals_conceded) / 2
        else:  # Forwards
            return 0
    
    elif game_type == '7-a-side':
        if player_position == 'Goalkeeper':
            return defense_score_7 - goals_conceded + goalkeeper_score_7
        elif player_position == 'Defensive Player':
            return defense_score_7 - goals_conceded
        else:  # Offensive Players
            return 0
    
    elif game_type == '5-a-side':
        if player_position == 'Goalkeeper':
            return defense_score_5 - goals_conceded + goalkeeper_score_5
        else:  # Outfield Players
            return defense_score_5 - goals_conceded

# Apply the function to the new_points_data DataFrame to compute the Defensive Score
new_points_data['Defensive Score'] = new_points_data.apply(calculate_defensive_score, axis=1)

def calculate_midfield_score(row):
    # Extracting necessary data from the row
    game_type = row['Game Type']
    player_position = row['Player Position']
    goals_conceded = row['Team A Goals'] if row['Team'] == 'Team B' else row['Team B Goals']
    goals_scored = row['Team B Goals'] if row['Team'] == 'Team A' else row['Team A Goals']
    
    # Define midfield score based on game type and player position
    if game_type == '11-a-side':
        if player_position in ['Goalkeeper', 'Defender']:
            return 0
        elif player_position == 'Midfielder':
            return math.ceil(goals_scored - goals_conceded)
        else:  # Forwards
            return math.ceil(goals_scored - goals_conceded) / 2
    
    elif game_type == '7-a-side':
        if player_position in ['Goalkeeper', 'Defensive Player']:
            return 0
        else:  # Offensive Players
            return math.ceil(goals_scored - goals_conceded)
    
    elif game_type == '5-a-side':
        if player_position == 'Goalkeeper':
            return 0
        else:  # Outfield Players
            return math.ceil(goals_scored - goals_conceded)

# Apply the function to the new_points_data DataFrame to compute the Midfield Score
new_points_data['Midfield Score'] = new_points_data.apply(calculate_midfield_score, axis=1)


In [4]:
# Save new_points_data to an Excel file
new_points_data.to_excel("new_points_data_output.xlsx", index=False)